In [2]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
import os
from openai import OpenAI 

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def text_to_speech(voice_path: str, text: str) -> bytes:
    response = client.audio.speech.create(model="tts-1", voice="nova", input=text)
    response.stream_to_file(voice_path)
    return response.content

In [26]:
voice_path = os.path.join("./", "love.mp3")

tts = text_to_speech(voice_path, "엄마가 현지를 매우 많이 사랑해")

/var/folders/9w/gtnfkqzx1rv_kh823gf0vq_c0000gn/T/ipykernel_89975/3100096720.py:8: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [27]:
from IPython.display import Audio, display

display(Audio(tts, autoplay=True))

In [28]:
def speech_to_text(voice_path):
  audio_file = open(voice_path, "rb")
  transcription = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file
  )
  return transcription.text

In [29]:
print(speech_to_text(voice_path))

엄마가 현지를 매우 많이 사랑해.


In [30]:
from string import Template

system_prompt = """
너는 사용자의 요청에 상담을 해주는 인공지능 에이전트야.
"""

user_prompt = Template("""
다음 사용자의 요청에 대한 답변을 해줘

요청: $query
""")

In [32]:
class VoiceChatbot():
  def __init__(self, voice_path, system_prompt, user_prompt):
    self.voice_path = voice_path
    self.file_id = 10
    self.system_prompt = system_prompt
    self.user_prompt = user_prompt
  
  def query(self, query_path):
    question = self.speech_to_text(query_path)
    answer = self.generate_answer(question)
    answer = self.post_process(answer)
    save_path = self.text_to_speech(answer)
    return save_path

  def speech_to_text(self, voice_file):
    audio_file = open(voice_file, "rb")
    transcription = client.audio.transcriptions.create(
      model="whisper-1",
      file=audio_file
    )
    return transcription.text

  def text_to_speech(self, txt):
    save_path = os.path.join(self.voice_path, f"{self.file_id}.mp3")
    response = client.audio.speech.create(
      model="tts-1",
      voice="echo",
      input=txt
    )
    response.stream_to_file(save_path)

    display(Audio(save_path, autoplay=True))
    self.file_id += 1

    return save_path

  def generate_answer(self, question):
    chat_completion = client.chat.completions.create(
    messages=[
            {
                "role": "system",
                "content": self.system_prompt,
            },
            {
                "role": "user",
                "content": self.user_prompt.substitute(query=question),
            }
        ],
        model="gpt-4o-mini",
    )
    response = chat_completion.choices[0].message.content

    return response

  def post_process(self, txt):
    return txt.replace("\n", "")


voiceChatbot = VoiceChatbot(
    voice_path="./",
    system_prompt=system_prompt,
    user_prompt=user_prompt
)

In [34]:
save_path = voiceChatbot.text_to_speech("안녕하세요. LLM을 활용한 AI 애플리케이션은 어떻게 개발하면 좋을까요?")

/var/folders/9w/gtnfkqzx1rv_kh823gf0vq_c0000gn/T/ipykernel_89975/3192878007.py:30: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(save_path)


In [35]:
response_save_path = voiceChatbot.query(save_path)

/var/folders/9w/gtnfkqzx1rv_kh823gf0vq_c0000gn/T/ipykernel_89975/3192878007.py:30: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(save_path)


### Gradio로 UI에서 대화하기

In [36]:
%pip install -q gradio

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-cli 0.0.28 requires tomlkit<0.13.0,>=0.12.2, but you have tomlkit 0.12.0 which is incompatible.
langchain-cli 0.0.28 requires typer[all]<0.10.0,>=0.9.0, but you have typer 0.12.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [37]:
import gradio as gr

ui = gr.Interface(
    fn=voiceChatbot.speech_to_text,
    inputs=gr.Audio(label="음성으로 질문하기", sources=["microphone"], type="filepath"),
    outputs="text",
)

ui.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [39]:
ui = gr.Interface(
      fn=voiceChatbot.query,
      inputs=gr.Audio(label="음성으로 질문하기", sources="microphone", type="filepath"),
      outputs=gr.Audio(label="음성으로 응답하기", type="filepath"),
  )

ui.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


/var/folders/9w/gtnfkqzx1rv_kh823gf0vq_c0000gn/T/ipykernel_89975/3192878007.py:30: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(save_path)
